# Phoneme Analysis

Comparing phonemes outputted by phonemization to phonemes stored in [phoible](https://phoible.org/). This is used to improve the phonemization by adding folding dictionaries to increase the coherence of the resulting phoneme set for each language.

In [1]:
import pandas as pd

from tqdm import tqdm

phoible = pd.read_csv('../../../data/phoible.csv')
phonemes = phoible.Phoneme.unique()

/var/folders/nz/6tzh0bsj2txd1cz18gpcms_c0000gn/T/ipykernel_50799/3962830865.py:5: DtypeWarning: Columns (4,7,8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  phoible = pd.read_csv('../../../data/phoible.csv')


In [2]:
def find_unseen_phonemes(counts, phonemes):
    """ Finds phonemes in counts unknown in the phonemes list, and phonemes in the phonemes list not seen in counts """
    unseen = set() 
    unknown = {} 
    for phoneme in phonemes:
        if phoneme not in counts:
            unseen.add(phoneme)
    for phoneme, count in counts.items():
        if phoneme not in phonemes and phoneme != 'WORD_BOUNDARY':
            unknown[phoneme] = count
    unknown = {k: v for k, v in sorted(unknown.items(), key=lambda item: item[1], reverse=True)}
    return unseen, unknown

def find_phoneme(csv_file, phoneme, count=1):
    """ Find lines where phoneme appears and also print gloss """
    dataset = pd.read_csv(csv_file)
    phonemized = list(dataset['phonemized_utterance'])
    for i, line in enumerate(phonemized):
        if phoneme in line:
            print(f'Line {i}: {dataset["gloss"][i]} - {line}')
            count -= 1
        if count == 0:
            break

def find_words_with_phoneme(csv_file, phoneme, single_phoneme=False):
    words_counts = {}
    dataset = pd.read_csv(csv_file)
    phonemized = list(dataset['phonemized_utterance'])
    glosses = list(dataset['gloss'])
    for i, line in tqdm(enumerate(phonemized)):
        if phoneme in line:
            words = glosses[i].split()
            phonemic_words = line.split(' WORD_BOUNDARY ')
            for j, word in enumerate(phonemic_words):
                if (single_phoneme and phoneme in word.split()) or (not single_phoneme and phoneme in word):
                    if j >= len(words):
                        continue
                    entry = words[j] + '/' + word.replace(' WORD_BOUNDARY', '')
                    if entry not in words_counts:
                        words_counts[entry] = 0
                    words_counts[entry] += 1
    # Sort by counts
    words_counts = {k: v for k, v in sorted(words_counts.items(), key=lambda item: item[1], reverse=True)}
    return words_counts

def print_unseen_phonemes(unseen, unknown):
    print(f'Unknown phonemes: {unknown if unknown else "None"}')
    print(f'Unseen phonemes: {unseen if unseen else "None"}')

def compare_phoneme_sets(csv_file, iso_code, language_names, inventory_id=None):
    """ Compares phonemes in the csv file with the phonemes in phoible.
    
    Four levels of comparison are made:
    1. Phonemes in the dataset not seen in any parts of phoible
    2. Phonemes in the dataset not seen in any phoible language with a specific language code (e.g. 'eng' for English)
    3. Phonemes in the dataset not seen in the specific inventories of phoible that match the language names (e.g. ['American English', 'English (American)'] for English)
    4. (Optional) Phonemes in the dataset not seen in a specific inventory of phoible (e.g. '2175' for English (American))
    """
    if not csv_file.endswith('.csv'):
        dataset = pd.read_csv('../CHILDES-dataset/'+csv_file+'/train.csv')
        dataset = pd.concat([dataset, pd.read_csv('../CHILDES-dataset/'+csv_file+'/valid.csv')])
    else:
        dataset = pd.read_csv(csv_file)

    token_counts = {}
    for line in dataset['phonemized_utterance']:
        # Our tool combines tone markers with the preceeding vowel, we remove tone markers in our comparison so that we don't get many "unknown phonemes" consisting of a known vowel + tone marker.
        line = line.replace('˧˥', '').replace('˧˩̰', '').replace('˩˧', '').replace('˨', '').replace('˥', '').replace('˧', '').replace('˧˥', '').replace('˧˩̰', '').replace('˩˧','').replace('˩','').replace('˦','')
        tokens = line.strip().split()
        for token in tokens:
            if token not in token_counts:
                token_counts[token] = 0
            token_counts[token] += 1

    print(f'Total phonemes in the dataset: {sum(token_counts.values())}')
    print(f'Unique phonemes: {list(token_counts.keys())}\n')

    _, unknown = find_unseen_phonemes(token_counts, phonemes)
    print('-'*100)
    print('ALL PHOIBLE:')
    print('Comparing the phonemes seen in the data with all phoible phonemes:')
    print(f'Unknown phonemes: {unknown if unknown else "None"}')


    language_phonemes = phoible[phoible.ISO6393 == iso_code].Phoneme.unique()
    language_unseen, language_unknown = find_unseen_phonemes(token_counts, language_phonemes)
    print('-'*100)
    print('LANGUAGE CODE:')
    print(f'\nComparing the phonemes seen in the data with just the phonemes in phoible with the {iso_code} language code:')
    print_unseen_phonemes(language_unseen, language_unknown)
    
    possible_language_names = phoible[phoible.ISO6393 == iso_code].LanguageName.unique()
    print(f'\nPhoible inventories names with the {iso_code} language code: {str(list(possible_language_names))}')

    possible_dialects = phoible[phoible.ISO6393 == iso_code].SpecificDialect.unique()
    print(f'Phoible dialects names with the {iso_code} language code: {str(list(possible_dialects))}')

    print('-'*100)
    print('INVENTORIES:')

    all_inventory_phonemes = set()
    inventory_ids = set()
    for inventory in language_names:
        dialect_phonemes = phoible[phoible.LanguageName == inventory].Phoneme.unique()
        ids = phoible[phoible.LanguageName == inventory].InventoryID.unique()
        all_inventory_phonemes.update(dialect_phonemes)
        inventory_ids.update(ids)

    dialect_unseen, dialect_unknown = find_unseen_phonemes(token_counts, all_inventory_phonemes)
    print(f'\nNow comparing the phonemes seen in the data with the phonemes in phoible with the specific language names {language_names}:')
    print(f'Inventory IDs matching the inventory names: {str(inventory_ids)}')
    print_unseen_phonemes(dialect_unseen, dialect_unknown)

    if inventory_id is not None:
        inventory_phonemes = phoible[phoible.InventoryID == inventory_id].Phoneme.unique()
        inventory_unseen, inventory_unknown = find_unseen_phonemes(token_counts, inventory_phonemes)
        print('-'*100)
        print(f'INVENTORY ID:')
        print(f'\nNow comparing the phonemes seen in the data with the phonemes in phoible with the specific inventory {inventory_id}:')
        print(f'Phonemes in inventory:', inventory_phonemes)
        print_unseen_phonemes(inventory_unseen, inventory_unknown)


# English (US) Analysis

The English section of our corpus is taken from Eng-NA in CHILDES, so we compare to all English languages and more specifically the two American English sections of phoible. For English, we use phonemizer.

The output of espeak, adjusted by our folding dictionary, brings the phoneme inventory to be pretty close to the American English phoneme inventories (specifically invenetory 2175) in phoible. The unseen phonemes mostly overlap with the unknown phonemes, with slight altered diacritics.

For English, we do not try to match some of the remaining unseen phonemes with the unknown phonemes, as our folding dictionary is based on the phonemizing processed used by [BabySLM](https://github.com/MarvinLvn/BabySLM) and we need to stick to the same transcription process if we want our datasets to match theirs for training and evaluation. 

In [3]:
compare_phoneme_sets('Eng-NA', 'eng', ['English (American)', 'American English'], 2175)

Total phonemes in the dataset: 40979962
Unique phonemes: ['s', 'iː', 'WORD_BOUNDARY', 'ð', 'ɛ', 'ɹ', 'z', 'ʌ', 'f', 'eɪ', 'w', 'ɪ', 'ɡ', 'l', 'æ', 'ɑ', 'h', 'ə', 'ʊ', 'k', 'p', 'uː', 'b', 'i', 't', 'aɪ', 'θ', 'ŋ', 'j', 'ɔ', 'm', 'ɔɪ', 'n', 'd', 'oʊ', 'aʊ', 'v', 'ɜː', 't̠ʃ', 'd̠ʒ', 'ʃ', 'iə', 'ʒ', 'x']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the eng language code:
Unknown phonemes: None
Unseen phonemes: {'eə', 'ɛː', 'əʉ', 'a', 'øː', 'əʊ', 'oɪ', 'aː', 'ɚː', 'u', 'ɐʉ', 'æo', 'kx', 'eː', 'ɐ', 'o̞ː', 'ɵː', 'kʰ', 'tʰ', 'ei', 'ɚ', 'iɪ', 'əː', 'iɛ', 'ɘ', 'æɔ', 'e̞', 'ɻ', 'oe', 'ɛʉ', 'ʍ', 'ɒɯ', 'oː', 'ɪə', 'ʊə', 'ɔː', 'æɪ', 'ʉː', 'æe', 'e',

# English (UK) Analysis

The English section of our corpus is taken from Eng-UK in CHILDES, so we compare to all English languages and more specifically the British English sections of phoible. We use the `en-gb` espeak accent which uses RP, so we compare to inventory 2252 (English RP). Note that much of the data comes from areas of the UK that do not use RP. 

With a few adjustments in our folding dictionary, we get a pretty good match to inventory 2252. There are just two extra vowels that the phonemizer outputs (`i`, and `ɐ`) as well as syllabic `n̩`. The rest of the unknown phonemes are sufficiently rare.

In [4]:
compare_phoneme_sets('Eng-UK', 'eng', ['English (British)', 'English'], 2252)

Total phonemes in the dataset: 28737383
Unique phonemes: ['eɪ', 't̠ʃ', 'WORD_BOUNDARY', 'w', 'ɒ', 'tʰ', 'd', 'ʌ', 'z', 'ð', 'æ', 'm', 'iː', 'n', 'aɪ', 'h', 'e', 'kʰ', 's', 'ɪ', 'ɡ', 'ʊ', 'ə', 'ɑː', 'ɔː', 'l', 'ɹ', 'i', 'əʊ', 'uː', 'j', 'ɪə', 'ɔɪ', 'v', 'f', 'ɜː', 'b', 'pʰ', 'd̠ʒ', 'ɐ', 'eə', 'ʃ', 'θ', 'ŋ', 'aʊ', 'ʊə', 'n̩', 'ʒ', 'ɔːɹ', 'x']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: {'ɔːɹ': 2}
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the eng language code:
Unknown phonemes: {'n̩': 3290, 'ɔːɹ': 2}
Unseen phonemes: {'ɛː', 'əʉ', 'a', 'øː', 'oɪ', 'aː', 'ɚː', 'k', 'u', 'ɐʉ', 'æo', 'kx', 'eː', 'oʊ', 'ɔ', 'o̞ː', 'ɵː', 't', 'ei', 'ɚ', 'iɛ', 'iɪ', 'əː', 'ɘ', 'æɔ', 'e̞', 'ɻ', 'oe', 'ɛʉ', '

# German Analysis

We compare all German inventories in phoible to our German dataset. For German, we use the epitran backend.

With our folding dictionary, we are close to inventory 2398. 

The phoneme `ʒ` is unseen and the unknown phonemes `x` and `ɐ` are missing from inventory 2398 but exist in other german inventories. The rest are sufficiently rare.


In [5]:
compare_phoneme_sets('German', 'deu', ['German','GERMAN'], 2398)

Total phonemes in the dataset: 27267742
Unique phonemes: ['aː', 'WORD_BOUNDaRY', 'oː', 'a', 'b', 'x', 'v', 'øː', 'n', 'ɛː', 'f', 'l', 'iː', 'yː', 'j', 'uː', 'h', 'ʊ', 'm', 'ɔ', 'ɪ', 'eː', 'ə', 'd̺', 't̺ʰ', 'ɛ', 'ŋ', 'ç', 'œ', 'kʰ', 'ʀ', 'ɡ', 'pʰ', 'ʏ', 's', 'z', 'ts', 'ʃ', 'ɐ', 'pf', 't̠ʃ', 'd̠ʒ']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: {'WORD_BOUNDaRY': 5825166}
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the deu language code:
Unknown phonemes: {'WORD_BOUNDaRY': 5825166}
Unseen phonemes: {'ʏː', 'e', 'ɔi', 'd', 'ʒ', 'au', 'p', 'tʰ', 't', 'ai', 'ɔɪ', 'k', 'aɪ', 'ʁ', 'ʔ', 'aʊ', 'y'}

Phoible inventories names with the deu language code: ['German', 'GERMAN']
Phoible dialects names 

# Japanese Analysis

We compare all Japanese inventories in phoible to our Japanese dataset. For Japanese, we use phonemizer.

The three phoible Japanese inventories do not seem to accurately reflect the Wikipedia page on Japanese phonology, but we get close to inventory 2196. There are few unseen phonemes `ɴ` and `u`. Since phonemizer uses a segments backend, we do not output `ɴ` in the correct contexts, but it is a rather infrequent phoneme. `ɯː` might be being output instead of `u`. 

There are many unseen phonemes, many of which are valid phonemes according to the Wikipedia entry and appear in some of the other inventories. 



In [6]:
compare_phoneme_sets('Japanese', 'jpn', ['Japanese', 'JAPANESE'], 2196)

Total phonemes in the dataset: 14956403
Unique phonemes: ['o', 'ts', 'ɯ', 'k', 'a', 'i', 'WORD_BOUNDARY', 'w', 'd̠ʒ', 't', 'e', 'n', 'ʃ', 'd', 'b', 'aː', 's', 'm', 'h', 'ɾ', 't̠ʃ', 'ɯː', 'p', 'j', 'ɡʲ', 'ɸ', 'ɡ', 'oː', 'ɲ', 'z', 'eː', 'pʲ', 'ç', 'ɾʲ', 'kʲ', 'bʲ', 'mʲ']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the jpn language code:
Unknown phonemes: {'oː': 223829, 'eː': 35711, 'ɸ': 31401, 'kʲ': 16378, 'ɲ': 9056, 'ɾʲ': 3774, 'ɡʲ': 3266, 'ç': 1978, 'pʲ': 1629, 'bʲ': 1482, 'mʲ': 220}
Unseen phonemes: {'tː', 'ɛː', 's̪', 'kː', 'u', 'd̠', 'ŋ', '˧', 'iː', 'ɔ', '˥', 'd̪', 't̪', 'n̪', 'ɯ̃', 'tsː', 'çː', 'pː', 'ɔː', 'ɴ', 'sː', 'ɛ', 'z̪', 'ʃː'

# Indonesian Analysis

We compare all Indonesian inventories in phoible to our Indonesian dataset. For Indonesian, we use the epitran backend. With minimal folding, we are close to inventory 1690.

The only unknown phoneme is `j`, which appears in the other Indonesian inventory. There are a few unseen phonemes, `e`, `ʔ`, `ɛ`, `v` and `y`. Three of these are not listed in the other inventory.



In [7]:
compare_phoneme_sets('Indonesian', 'ind', ['Indonesian'], 1690)

Total phonemes in the dataset: 11718625
Unique phonemes: ['n', 'i', 'h', 'WORD_BOUNDARY', 'l', 'o', 't', 'm', 'a', 'w', 's', 'd̠ʒ', 'ŋ', 'ə', 'd', 'p', 'ɡ', 'b', 'u', 'r', 'k', 'ɲ', 'j', 't̠ʃ', 'f', 'z', 'ʃ', 'x']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the ind language code:
Unknown phonemes: None
Unseen phonemes: {'ɛ', 't̪', 'ai̯', 'oi̯', 'ʔ', 'v', 'e', 'y', 'au̯'}

Phoible inventories names with the ind language code: ['Indonesian']
Phoible dialects names with the ind language code: ['Central Java', 'Standard']
----------------------------------------------------------------------------------------------------
INVENTORIES:

Now 

# French Analysis

We compare all French inventories in phoible to our French dataset. For French, we use phonemizer.

We find a few unknown phonemes, all very infrequent and most are valid allophones according to the French inventory:
* `yː` is an allophone of `y`
* `aː` is an allophone of `a̟`
* `iː` is an allophone of `i`

The remaining two phonemes `t̠ʃ` and `d̠ʒ` seem to come from loan words such as `sandwhich` and `jazz` so seem acceptable to keep. 

Comparing to the specific inventory ID `2266` there is only one unseen phoneme, `ɥ` and the unknown phonemes are either those mentioned above, rare, or contained in one of the other French inventories (such as `œ̃`).

In [8]:
compare_phoneme_sets('French', 'fra', ['French', 'FRENCH'], 2269)

Total phonemes in the dataset: 11176967
Unique phonemes: ['d', 'ɔ', 'n', 'WORD_BOUNDARY', 'b', 'a', 'm', 'ɑ̃', 'ʁ', 'ə', 'ɡ', 'ʒ', 'i', 'v', 't', 'k', 'o', 'ɛ̃', 'w', 'y', 'j', 'e', 'ɔ̃', 'p', 'ɛ', 'f', 's', 'z', 'l', 'u', 'ʃ', 'œ', 'ø', 'ɲ', 't̠ʃ', 'd̠ʒ', 'ŋ']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the fra language code:
Unknown phonemes: {'t̠ʃ': 48, 'd̠ʒ': 11}
Unseen phonemes: {'ɛː', 's̪', 'ʀ', 'æ̃', 'ø̞', 'k̟', 'ɡ̟', 'ɦ', 'œ̃', 'd̪', 't̪', 'n̪', 'a̟', 'ɥ', 'oː', 'õ', 'ɑ', 'r', 'ɒ̃', 'ɒ', 'z̪', 'l̪'}

Phoible inventories names with the fra language code: ['French', 'FRENCH']
Phoible dialects names with the fra language code: [n

# Spanish Analysis

We compare all Spanish inventories in phoible to our Spanish dataset. For Spanish, we use epitran.

With our folding dictionary, we get close to inventory 164. The unseen phonemes `θ` and `ʎ` do not always appear in Spanish phonology, depending on accent. `ʝ` is unknown but typically replaces `ʎ` in certain accents. `d` and `g` are unseen for this inventory but are listed in others. `ʃ` is unknown but mostly comes from english loan words. The remaining unknown phonemes are sufficiently rare. 


In [9]:
compare_phoneme_sets('Spanish', 'spa', ['Spanish', 'SPANISH'], 164)

Total phonemes in the dataset: 9926542
Unique phonemes: ['i', 'ɾ', 'e̞', 'n', 'WORD_BOUNDARY', 'k', 'a', 'ɲ', 'o̞', 'm', 's', 'u', 'p', 'd', 'l', 't', 'β', 'ɡ', 'w', 'ʝ', 'f', 'x', 'j', 'r', 't̠ʃ', 'ʃ', 'tl', 'ts']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the spa language code:
Unknown phonemes: {'d': 234418, 'ʝ': 46455, 'tl': 139, 'ts': 90}
Unseen phonemes: {'θ', 'u̯æi', 's̪', 'ð͉', 'i̯e', 'i̯a', 'ue', 'u̯a', 'ai̯', 'i̯o', 'i̯', 'i̯ɔ', 'eu̯', 'oi', 'i̯ɛ', 'ɔ', 'æ', 'au', 'ɣ', 'uei̯', 'ai', 'ei', 't̪', 'd̪', 'd̻', 'u̯ɛi', 'ua', 'n̪', 't̻', 'ɟʝ', 'ou̯', 'ui', 'χ', 'æi', 'ɟ', 'ð', 'b', 'ei̯', 'ɔi', 'ia', 'o', 'i̯æi', 'ie', 's̺̠', 'ʎ',

# Mandarin Analysis

We compare all Mandarin inventories in phoible to our Mandarin dataset. For Mandarin, we use pinyin_to_ipa.

There are four Mandarin inventories in Phoible, "Mandarin Chinese", "Standard Chinese; Mandarin", "MANDARIN" and "Standard Chinese". There is considerable disagreement between these.

Our output seems to mostly align with the inventory code 2457 (Beijing Standard Chinese) with the help of a few folding rules to mach diacritics used by the inventory (e.g. `au` instead of `au̯`).

Note that our tool (and pinyin_to_ipa) combines tone markers with the preceeding vowel, which is not how phoible lists tone markers. We remove tone markers in our comparison so that we don't get many "unknown phonemes" consisting of a known vowel + tone marker.

There are a few key unknown phonemes:
* `o`, `ɛ`, `ʊ`, `e`, `ɔ`: these are vowels not listed in the inventory but are typically used in transcriptions as allophones of typical Mandarin phonemes. pinyin_to_ipa seems to favour a slihgly more phonetic realisation of vowels.
* `ɻ̩` and `ɻ`: listed in other sources as a valid consonant (and syllabic consonant) in Mandarin but not present in this inventory
* `ɥ`: a glide used along with `j` and `w` in certain analyses of Mandarin, this inventory seems to prefer to write these as diphthongs (see below). 
* `ɚ`: the rhotic coda present in the Beijing dialect for a few words but not listed in this inventory. 

The unseen tokens consist of:
* `ɹ̺`: It is unclear why this is not produced
* `ua`, `iau`, `iu`, `uei`, `ye`, `uo`, `uai`, `ie`, `uə`, `ia`, `iou`: dipthongs resulting from the alternative analysis of glides where `j`, `ɥ` and `w` are not analysed as independent phonemes but instead as allophones of the high vowels `i`, `y` and `u`. Although this intepretation may be more common, we stick to the output of pinyin_to_ipa rather than try to correct this. 

In [10]:
compare_phoneme_sets('Mandarin', 'cmn', ['Standard Chinese; Mandarin', 'Mandarin Chinese'], 2457)

Total phonemes in the dataset: 8870431
Unique phonemes: ['a', 'u', 'WORD_BOUNDARY', 'au', 'n', 'ʃ̺', 'ɻ̩', 'ə', 'm', 'ɤ', 'p', 'j', 'e', 'kʰ', 'k', 'w', 'o', 't̠ʃ̺ʰ', 'ŋ', 't', 'ʊ', 'ɕ', 'i', 'l', 'x', 'ei', 'pʰ', 'ai', 'ou', 'tɕ', 'ts', 't̠ʃ̺', 's', 'ɹ̪̩', 'tɕʰ', 'ɛ', 'f', 'y', 'ɻ', 'tʰ', 'ɥ', 'ɔ', 'tsʰ', 'ɚ']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the cmn language code:
Unknown phonemes: {'o': 144716, 'ɻ̩': 138241, 'ɛ': 79987, 'ʊ': 53095, 'e': 45152, 'ɻ': 37888, 'ɔ': 12193, 'ɚ': 5432}
Unseen phonemes: {'t̪s̪ʰ|tsʰ', 'w˞', 'cç', 'ɤ˞', 'ʈʂʰ', '˧˦', 'iou', 'ə̃˞', 'l̪|l', '˧˥', 'ɰ', '˧˨˥', '˦˨', 'ə˞', 'ʂ', 't̪|t', 'ɹ', 'cçʰ', 'uei', 

# Dutch Analysis

We compare all Dutch inventories in phoible to our Dutch dataset. For Dutch, we use phonemizer.

With our folding map, we get close to inventory 2405, although there are a few major differences.

The inventory lists `χ` as unseen, but instead we produce `ɣ` and `x` which are listed on Wikipedia. The inventory also lists `ɔ̃`, `ɛ̃`, `œː` and `ã` as missing. There are also many unknown phonemes. Many are rare, the others are mostly listed on Wikipedia or in other inventories.

In [11]:
compare_phoneme_sets('Dutch', 'nld', ['Dutch'], 2405)

Total phonemes in the dataset: 6261977
Unique phonemes: ['j', 'ãː', 'WORD_BOUNDARY', 'ɦ', 'oː', 'ɾ', 'd', 'i', 'ɛ', 'p', 'ɪ', 'k', 'ɑ', 'l', 'ɛː', 'n', 's', 'v', 'ə', 'ɛi', 'ʋ', 'z', 't', 'm', 'ɣ', 'ʏ', 'ɔ', 'x', 'u', 'f', 'ŋ', 'øː', 'b', 'ɔː', 'ʌu', 'y', 'œy', 'tʲ', 'w', 'ʃ', 't̠ʃ', 'ɲ', 'ʒ', 'iː', 'ɡ', 'd̠ʒ', 'ã', 'uː', 'yː']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the nld language code:
Unknown phonemes: {'ãː': 228246, 't̠ʃ': 261}
Unseen phonemes: {'eə', 'eɪ', 'pʲ', 'a', 'əʊ', 'ʀ', 'aː', 'ø', 'ɔuː', 'ɑu', 'ɑ̃ː', 'c̠', 'oə', 'ɑuː', 'ɛi̯', 'eː', 'kʲ', 'h', 'c', 'æ̃ː', 'uɪ', 'æ', '˦', 'øy', 'ɛə', 'ei', 'χ', '˨', 'ʊ', 'ɟ', 'yːə',

# Serbian Analysis

We compare the Serbian inventory in phoible to our Serbian dataset. For Serbian, we use epitran. 

With our folding dictionary, we get close to inventory 2499 (the only inventory for Serbian).

The unseen phonemes are mostly long vowels, which epitran fails to produce (**this might be an issue**).

The unknown phoneme `j` seems to be missing from phoible and is a common phoneme in Serbian, according to wikipedia. The remaining unknown phonemes are rare and come from english loan words that have been badly transcribed.


In [12]:
compare_phoneme_sets('Serbian', 'srp', ['Serbian'], 2499)

Total phonemes in the dataset: 4893937
Unique phonemes: ['d̪̻', 'ä', 'WORD_BOUNDARY', 'i', 'j', 'm', 's̪̻', 'l', 'n', 'e̞', 'ʋ', 'r', 'o̞', 't̪̻', 'k', 't̪̻s̪̻', 'p', 'u', 'ʃ̺', 'x', 'b', 'ʒ̺', 'ɡ', 't̻ʃ̻', 'f', 'z̪̻', 'ɲ', 'ʎ', 'd̻ʒ̻', 'y', 'w']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the srp language code:
Unknown phonemes: {'j': 179606, 'w': 50, 'y': 46}
Unseen phonemes: {'t̺ʃ̺ʷ', 'o̞ː', 'äː', 'uː', 'e̞ː', 'd̺̠ʒ̺ʷ', 'iː'}

Phoible inventories names with the srp language code: ['Serbian']
Phoible dialects names with the srp language code: ['Serbian (Standard Ekavian)']
-----------------------------------------------------------

# Estonian Analysis

We compare all Estonian inventories in phoible to our Estonian dataset. For Estonian, we use phonemizer.

To create our folding dictionary, we compare our output with phoible inventory 2181 and Wikipedia. Many changes were required in the folding dictionary, due to repeated letters for long vowels and consonants being produced by phonemizer.

In the end, we have quite a bit of disagreement between our output and phoible inventory 2181 since the inventory lists semi-long consontants (which phonemizer does not output) and our tool outputs many consonants, long vowels and diphthongs that are not listed in the phoible inventory (but seem to be valid according to Wikipedia).

Generally, we get a pretty good match with the Wikipedia entry for Estonian phonology, except for the fact that we do not output many diphthongs, instead many vowels are listed independently (e.g. `e o` instead of `eo`). We could add rules to join these vowels but it is not clear if these should always be diphthongs, so we leave them. 

However, looking at individual examples, the output is not always very good (e.g. initial `b` being output as `p` or `i` randomly being added after some other vowels). The espeak-ng voice might not be very good for this language.


In [13]:
compare_phoneme_sets('Estonian', 'ekk', ['Estonian'], 2181)

Total phonemes in the dataset: 4272417
Unique phonemes: ['n', 'o', 'WORD_BOUNDARY', 'm', 'i', 's', 'a', 'æ', 'r', 'p', 'l', 'e', 'd', 'iː', 't', 'ʃ', 'kː', 'v', 'æi', 'pː', 'u', 'k', 'h', 'mː', 'eː', 'sː', 'ɑ', 'yː', 'uː', 'ɛ', 'æː', 'j', 'ɡ', 'b', 'aː', 'sʲ', 'ɪ', 'ɤ', 'ɔ', 'ɤː', 'ʊ', 'lː', 'ø', 'øː', 'tː', 'ŋ', 'y', 'tʲ', 'oː', 'rː', 'ɲ', 'nː', 'w', 'tʲː', 'øɪ̯', 'f', 'dʲ', 'sʲː', 't̠ʃ', 'ʃː', 'ʒ', 'z', 'fː', 'dː', 'yi', 'jː']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the ekk language code:
Unknown phonemes: {'a': 356082, 'd': 106993, 'iː': 88836, 'ɡ': 53705, 'aː': 52949, 'eː': 48995, 'ɛ': 48895, 'b': 37424, 'ɪ': 25300, 'ʊ': 20865,

# Cantonese Analysis

We compare all Cantonese inventories in phoible to our Cantonese dataset. For Cantonese, we use pingyam.

There are four Cantonese inventories in Phoible, "Chinese", "Cantonese", "TAISHAN" and "Yue Chinese". 

Our output closely aligns with inventory 2166. The only unknown phoneme is syllabic m (`m̩`) and the only unseen phonemes are `kʷʰ`, `r`, `kʷ` and `β`.

In [14]:
compare_phoneme_sets('Cantonese', 'yue', ['Cantonese', 'TAISHAN', 'Yue Chinese', 'Chinese'], 2166)

Total phonemes in the dataset: 2642768
Unique phonemes: ['a̞', 'WORD_BOUNDARY', 't', 'ɐ', 'k', 'l', 'j', 'ʊ̟', 'ɛ', 'n', 'ei', 'w', 'ɐi', 'm̩', 'm', 'ou', 'i', 'ts', 'ɔ̽', 'tʰ', 'f', 'aːĭ', 'p', 'h', 'ɵy', 'u', 'ŋ', 's', 'ɔːĭ', 'ɐu', 'ɪ̞', 'iːŭ', 'ɵ', 'tsʰ', 'kʰ', 'aːŭ', 'pʰ', 'y', 'œ̞', 'uːĭ']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the yue language code:
Unknown phonemes: None
Unseen phonemes: {'ʃ', 'a', 'l̪|l', 't̠ʃʰ', '˧˥', 'kʷʰ', '˧', 't̪|t', 'ŋ̩', 'n̪|n', 'ɛu', 'ɔ', 'l̥', 'æ', '˦', 'au', 'ai', '˥', 'ui', 'ɪ', '˨', 'ʊ', 'ɔi', 'β', '˧˩', 't̪ʰ|tʰ', 'r', 'l̪̥|l̥', 'kʷ', '˩˧', '˧˩̰', 't̠ʃ', 'œ', 'iu', '˥˦', '˩'}

Phoible inve

# Polish analysis

We compare the two Polish inventories in phoible to our Polish dataset. For Polish, we use phonemizer. 

With some mapping, we get close to inventory 1046. 

The phoneme `d̪z̪` does not seem to be produced by phonemizer, but the other two unseen phonemes `c` and `ɟ` are not listed on Wikipedia so it seems ok to miss these.

The phonemes are tool produces that are not in the inventory ("unknown phonemes" below) are the consonants `t, d, ɡʲ, kʲ, xʲ` which are valid according to Wikipedia, as well as the vowel, `ẽ` which is listed in the other phoible set.


# 

In [15]:
compare_phoneme_sets('Polish', 'pol', ['Polish'], 1046)

Total phonemes in the dataset: 5404638
Unique phonemes: ['v', 'o', 'w', 'a', 'j', 'WORD_BOUNDARY', 'b', 'r', 'ɲ', 'i', 'ɕ', 'u', 'e', 'x', 'tɕ', 't̪', 'k', 'p', 'l̪', 'ɨ', 'dʑ', 'd̪', 'z̪', 'n̪', 'f', 'ʑ', 'm', 'z̻', 's̻', 't̻s̻', 't̪s̪', 'ɡ', 's̪', 'ŋ', 'kʲ', 't', 'ɡʲ', 'ẽ', 'ɣ', 'd̻z̻']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the pol language code:
Unknown phonemes: {'kʲ': 13194, 'ɡʲ': 3505, 'ẽ': 1430, 'ɣ': 1201, 't': 332}
Unseen phonemes: {'c', 'ĩ̯', 'ũ̯', 'ä', 'ɔ', 'ʂ̻', 'ɛ', 'd̪z̪', 'ʐ̻', 'ɖ̻ʐ̻', 'ʈ̻ʂ̻', 'ɘ̟', 'ɟ'}

Phoible inventories names with the pol language code: ['Polish']
Phoible dialects names with the pol langua

# Swedish analysis

We compare the two Swedish inventories in phoible to our Swedish dataset. For Swedish, we use phonemizer.

With our folding map, we get close to inventory 1150.

There are no unseen phonemes from the inventory and only a few unknown phonemes (`e, ʂ, ʃ, ɵː, z`) which are either rare or are listed as valid phonemes for Swedish on Wikipedia.

In [16]:
compare_phoneme_sets('Swedish', 'swe', ['Swedish'], 1150)

Total phonemes in the dataset: 2364143
Unique phonemes: ['ɔ', 'ʝ', 'WORD_BOUNDARY', 'k', 'l', 'ɛ', 'm', 'd̪', 'e', 'ʉ̟', 'f', 'ɪ', 'ŋ', 'ɹ', 'a', 'n̪', 'iː', 'ɑː', 'ɛː', 't̪', 's̪', 'v', 'oː', 'uː', 'eː', 'ʊ', 'p', 'b', 'h', 'øː', 'yː', 'ʂ', 'ɡ', 'ɵ', 'ʃ', 'œ', 'ɕ', 'ʏ', 'ɧ', 'z']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the swe language code:
Unknown phonemes: {'e': 32302, 'ʂ': 1304, 'ʃ': 1133, 'z': 13}
Unseen phonemes: {'kʰ', 'pʰ', 't̪ʰ', 'ʉ̟ː', 'l̪'}

Phoible inventories names with the swe language code: ['Swedish']
Phoible dialects names with the swe language code: ['Stockholm', 'Swedish (Central)']
-----------------------------

# Portuguese (portugal) analysis

We compare the Portuguese inventories in phoible to our Portuguese dataset. For Portuguese, we use phonemizer.

We need a substantial folding map, but by correcting the output we get very close to inventory 1150. However, this does require some items in the folding dictionary that do not simply change the symbol used for a particular phoneme, some other decisions also needed to be made.

With these decisions, all that remains is one rare unknown phonemes used for "pizza".

In [17]:
compare_phoneme_sets('PortuguesePt', 'por', ['Portuguese', 'Portuguese (European)'], 2206)

Total phonemes in the dataset: 2037930
Unique phonemes: ['p', 'a', 'ɐ', 'WORD_BOUNDARY', 'ɔ', 'l̪ˠ', 'i', 'n̪', 'e', 'ʃ', 'f', 'ɾ', 'ɐ̃', 'd̪', 'm', 'ʒ', 'b', 'ɯ', 'ɛ', 'ɐ̃i', 'ʁ', 't̪', 's', 'o', 'ɐ̃u̜', 'ũ', 'ɡ', 'u', 'k', 'z', 'au̜', 'ai', 'eu̜', 'ɐi', 'ɲ', 'ɛu̜', 'ĩ', 'ũi', 'õ', 'ɔi', 'õi', 'ẽ', 'v', 'oi', 'ʎ', 'iu̜', 'ui', 'ɛi', 'ts']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the por language code:
Unknown phonemes: {'ts': 55}
Unseen phonemes: {'w', 'ĕ', 'ŭ', 'iʊ̯', 'ɐ̃ɪ̯̃', 's̪', 's̻', 'ẽɪ̯̃', 'oʊ̯', 'ŋ', 'i̯', 'x', 'l', 'ĩ̯', 'ɛʊ̯', 'ɔɪ̯', 'ə', 'ɣ', 'z̻', 'aɪ̯', 'u̯', 'ũɪ̯̃', 'ɔʊ̯', 'n', 'ũ̯', 'aʊ̯', 'ɯ̽', 'oɪ̯', '

# Korean analysis

We compare the Korean inventories in phoible to our Korean dataset. For Korean, we use phonemizer.

Our system does not seem to output long vowels or diphthongs (this could be a problem) but inventory 423 does the same, so we use our folding map to approach this inventory.

Unknown phonemes:
* `b`, `d`, `dʑ` and `ɡ` are the voiced `p` `t` `tɕ` and `k` which are valid but not listed in the inventory for some reason
* `ɾ` is an allophone of `l` so should be valid

We have several unseen phonemes. Five are consonants with glottal stops or aspiration, which don't seem to be produced by our tool. The remaining unseen phonemes are vowels. It is unclear why these are not produced by our tool. 



In [18]:
compare_phoneme_sets('Korean', 'kor', ['Korean', 'KOREAN'], 423)

Total phonemes in the dataset: 1608306
Unique phonemes: ['w', 'a', 'WORD_BOUNDARY', 'j', 'o', 'ɾ', 'i', 'h', 'n̪', 'ɯ', 'd', 'e', 'ŋ', 'p', 'dʑ', 'u', 'm', 'ɡ', 'ɤ̞', 'l', 't̠ʃ', 'æ', 'b', 's̪', 'k', 't̪', 'pʰ', 'kʰ', 'ɯi', 't̠ʃʰ']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the kor language code:
Unknown phonemes: {'ɡ': 105270, 'd': 35819, 'dʑ': 19418, 'b': 7914}
Unseen phonemes: {'ɛ̝', 'ɛː', 't̠ʃˀ', 'ɤː', 'uɛ', 'ɡ̊', 't͉', 'aː', 'd̥ʑ̥', 'kˀ', 'ø', 's̪ˀ', 'ue', 'uʌ', 't̪ʰ', 'eː', 'iː', 's', 'sʰ', 'p͉', 'sˀ', 'ɐ', 's͉', 'ɤ', 'iɛ', 't', 'tʰ', 'uː', 'tɕ', 'əː', 'ua', 'ʌ', 'ui', 'ɑ̟ː', 'e̞', 'd̥', 'iʌ', 'ɪ', 'ʊ̜', 'y', 'n', 'tɕʰ', 'oː', '

# Italian analysis

We compare the Italian inventories in phoible to our Italian dataset. For Italian, we use phonemizer.

The inventory with the closest match is 1145 (which Wikipedia cites). 

We have no unseen phonemes but many unknown phonemes. These are:
* Long vowels: according to Wikipedia the distinction between long and short vowels is rare, but our tool seems to produce many long vowels. We could remove the length marker, but for now we keep it.
* Long consonants: according to Wikipedia, Italian consonants can geminate, so we keep these
* `ɾ` is an allophone for `r` so we keep it
* `ŋ` is an allophone for `n` so we keep it
* `ʒ` and `h` are very rare so we allow these




In [19]:
compare_phoneme_sets('Italian', 'ita', ['Italian'], 1145)

Total phonemes in the dataset: 1662350
Unique phonemes: ['ɐ', 'n', 'i', 'm', 'l', 'WORD_BOUNDARY', 'k', 's', 't', 'o', 'ɔ', 'z', 'f', 'v', 'e', 'd', 'j', 't̠ʃ', 'b', 'ɛ', 'w', 'ɛː', 'p', 'r', 'u', 'kː', 'ɡ', 'ʎ', 'd̠ʒ', 'tː', 'ɐː', 'pː', 'ts', 'dː', 'oː', 'iː', 'sː', 't̠ʃː', 'ɾ', 'eː', 'dz', 'bː', 'd̠ʒː', 'ɲ', 'tsː', 'ʃ', 'a', 'ɔː', 'dzː', 'ŋ', 'h', 'mː', 'uː', 'ɡː', 'vː', 'ʒ', 'fː']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the ita language code:
Unknown phonemes: {'tː': 15795, 'iː': 4538, 'sː': 3958, 'ɛː': 2508, 'ɐː': 1860, 'oː': 1272, 'ŋ': 716, 'tsː': 704, 'dː': 664, 'eː': 508, 'dzː': 157, 'uː': 28, 'ɔː': 26, 'h': 17}
Unseen phone

# Catalan analysis

We compare the Catalan inventories in phoible to our Catalan dataset. For Catalan, we use phonemizer.

With our folding dictionary we get close to inventory 2555. We have two unseen phonemes, `dz̺` and `i̯`. It is unclear why these are not produced. I'm also not sure what the distinction between `i` and `i̯` is. It is also worth noting that none of the Catalan inventories include diphthongs, nor does our tool produce diphthongs, even though Wikipedia says that there are phonemic diphthongs in Catalan.

The three unknown phonemes, `ð`, `β`, `ɣ` are the lenited versions of `d`, `b` and `g` which are produced in Catalan, so we keep these.


In [20]:
compare_phoneme_sets('Catalan', 'cat', ['Catalan'], 2555)

Total phonemes in the dataset: 1404320
Unique phonemes: ['a', 'WORD_BOUNDARY', 'w', 'ɛ', 'ə', 'ð', 't̪', 'j', 'i', 'ɔ', 'n̺', 'z̺', 'd̪', 's̺', 'β', 'm', 'e', 'f', 'ɾ̺', 'r̺', 'u̯', 'k', 'b', 'p', 'ɣ', 'ɡ', 'u', 'ŋ', 'o', 'ɫ̺', 'ɲ̟', 'ʒ', 'ʃ', 'ʎ̟', 't̠ʃ', 'd̠ʒ', 'ts̺']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the cat language code:
Unknown phonemes: {'β': 29440, 'ð': 17453, 'ɣ': 10169}
Unseen phonemes: {'ɾ', 'ɛu̯', 'i̯a', 'u̯a', 'dz̺', 'i̯', 'l̺', 's', 'l', 'c', 'i̯ɛ', 'ɔ̞', 'u̯ə', 'ou̯', 'a̟', 'v', 'ɛ̞', 'ə̹', 'ɟ', 'n', 'ei̯', 'ʎ', 'r', 'ui̯', 'z', 'ɲ'}

Phoible inventories names with the cat language code: ['Catalan']
Phoible dia

# Croatian analysis

We compare the Croatian inventory in phoible to our Croatian dataset. For Croatian, we use phonemizer.

With our folding dictionary, we get close to inventory 1139 (the only inventory for Serbian).

The unseen phonemes are mostly long vowels, which epitran fails to produce (**this might be an issue**). We also don't seem to produce `tɕ` and `dʑ`.

The unknown phonemes are sufficiently rare.

In [21]:
compare_phoneme_sets('Croatian', 'hrv', ['Croatian'], 1139)

Total phonemes in the dataset: 1414808
Unique phonemes: ['u', 'x', 'WORD_BOUNDARY', 'k', 'e', 'a', 't̪', 'n', 'o', 'd̪', 'i', 'r', 'm', 'ʃ', 'p', 's', 'ʋ', 'j', 't̠ʃ', 'l', 'ɡ', 'ʒ', 'b', 't̪s', 'z', 'd̠ʒ', 'ʎ', 'f', 'ɲ', 'y', 'q', 'w']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the hrv language code:
Unknown phonemes: {'y': 59, 'q': 42, 'w': 29}
Unseen phonemes: {'dʑ', 'oː', 'aː', 'tɕ', 'uː', 'ie', 'eː', 'iː'}

Phoible inventories names with the hrv language code: ['Croatian']
Phoible dialects names with the hrv language code: [nan]
----------------------------------------------------------------------------------------------------
I

# Welsh analysis

We compare the Welsh inventory in phoible to our Welsh dataset. For Welsh, we use phonemizer.

With our folding dictionary, we get close to inventory 1139 (the only inventory for Welsh).

Unseen phonemes
* `m̥`, `r̥`, `ŋ̥` and `n̥` - voiceless consonants which are not included in some interpretations of Welsh phonology.
* `tʰ`, `pʰ` and `kʰ` - aspirated stops, not always included in Welsh phonology.
* `t̠ʃ` - a rare phoneme mostly found in loan words.
* `əu` - a diphthong that should be produced, unclear why it is not here.

Unknown phonemes:
* `d`, `b`, `g` - voiceless stops. These should definitely be listed, not clear why the inventory does not include them. 
* `ø` and `ʌ` - vowels not listed in the inventory, it's not obvious what they should be mapped to.  They are both produced when there is a `y` in the orthography and should be different vowels in different contexts, so can't be easily mapped. In most cases, they should be `iː` or `ɪ`. **This may be an issue**

In [22]:
compare_phoneme_sets('Welsh', 'cym', ['Welsh'], 2406)

Total phonemes in the dataset: 2605636
Unique phonemes: ['ø', 'h', 'WORD_BOUNDARY', 'd', 'eː', 'χ', 'r', 'ai', 'ɛ', 't', 'ɔ', 'w', 'a', 'n', 'j', 'au', 'ə', 'ɔi', 'ð', 'oː', 'iː', 'ɪ', 'b', 's', 'ɡ', 'ʊi', 'ʊ', 'm', 'əi', 'θ', 'l', 'ʌ', 'ŋ', 'v', 'k', 'ɬ', 'p', 'f', 'ɪu', 'uː', 'ʃ', 'ɑː', 'ɛu', 'd̠ʒ', 'z']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the cym language code:
Unknown phonemes: {'d': 131028, 'ø': 52358, 'b': 47839, 'ɡ': 41191, 'ʌ': 32131}
Unseen phonemes: {'ŋ̥', 'kʰ', 'pʰ', 'tʰ', 'n̥', 't̠ʃ', 'r̥', 'm̥', 'əu'}

Phoible inventories names with the cym language code: ['Welsh']
Phoible dialects names with the cym language code:

# Icelandic analysis

We compare the Icelandic inventories in phoible to our Icelandic dataset. For Icelandic, we use phonemizer.

With our folding dictionary, we get close to inventory 2568.

Unseen phonemes:
* `aː`, `ɛː`, `ɔː`, `uː`, `iː`, `ɪː`, `ʏ` and `œː` - long vowels that should maybe be shortened, but mostly do exist in the other inventory.
* `ə` - not in the other inventory, unclear if it should be here
* `g` - not in either inventory but listed on wikipedia. Could be mapped to `kʰ`?
* `z` and `ʃ` which are sufficiently rare

Generally, it does not seem to be a great mapping.

In [23]:
compare_phoneme_sets('Icelandic', 'isl', ['Icelandic'], 2568)

Total phonemes in the dataset: 1337174
Unique phonemes: ['pʰ', 'iː', 'i', 'WORD_BOUNDARY', 'aː', 'r̥', 'ɪ', 'ɛ', 't̪ʰ', 's̺', 'j', 'ä', 'k', 'ʋ', 'ɛː', 'r', 'ei̯', 'θ̻', 'l', 'n̪', 't̪', 'ɬ', 'uː', 'ð̺̞', 'ɡ', 'c', 'h', 'ɔ', 'n̪̥', 'äu̯', 'ŋ̥', 'ʏ', 'ɔː', 'x', 'cʰ', 'm', 'f', 'ou̯', 'p', 'ŋ', 'øɪ̯', 'äi̯', 'ɰ', 'ʏː', 'u', 'ɪː', 'œ', 'ə', 'œː', 'ɲ', 'm̥', 'ç', 'ɔi̯', 'z', 'n̪ː', 'ɲ̥', 'ʏi̯', 'ʃ']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the isl language code:
Unknown phonemes: {'ɡ': 18371, 'ɪː': 8500, 'ə': 6532, 'ʏː': 1570, 'z': 12, 'ʃ': 4, 'n̪ː': 2}
Unseen phonemes: {'j̥', 'θ', 'ʍ', 'e', 'a', 'l̥', 'øː', 'ɣ', 'kʰ', 't', 'tʰ', 'ø'

# Danish analysis

We compare the Danish inventories in phoible to our Danish dataset. For Danish, we use phonemizer.

The output of phonemizer differs substantially from phoible and Wikipedia. With our folding dictionary we get close to inventory 2265, but there are some issues.

Unseen phonemes:
* `r` - phonemizer was producing `ɐ̯`, which is a correct phonetic realisation of `/r/`, but not listed in phoible, so we map it to the phoneme `/r/`. 
* `d`, `g`, `b`, `ŋ`, `ɕ`, `ð`, `w` - consonants not listed the other inventory (unclear why) but listed on Wikipedia. 
* `eˤ`, `ɑˤ`, `iˤ`, `uˤ`, `oˤ`, `aˤ`  - the Danish stød, which phonemizer seems to produce 
* `ɒ`, `ɑ`, `ɒː`, `uː`, `œː`, `ɜ`, `ɔː`, `oː` - vowels not listed in this inventory, but are listed in the other inventory or on Wikipedia
* Remaining phonemes are rare or from loan words

Unknown phonemes:
* `kʰ`, `pʰ` and `tsʰ` - aspirated consonants which the tool doesn't seem to produce
* `ø` - a vowel listed in this inventory that doesn't seem to be produced

Generally, it does not seem to be a great mapping.

In [24]:
compare_phoneme_sets('Danish', 'dan', ['Danish'], 2265)

Total phonemes in the dataset: 1099484
Unique phonemes: ['n', 'oˤ', 't', 'WORD_BOUNDARY', 'y', 'ə', 'ð', 'ʁ', 'ɑˤː', 's', 'k', 'i', 'b', 'eˤ', 't̠ʃ', 'a', 'l', 'd', 'ɡ', 'f', 'e', 'ɛ', 'r', 'ɔ', 'w', 'ɔˤ', 'm', 'uˤ', 'j', 'ɑ', 'u', 'ɒ', 'iˤ', 'ʋ', 'h', 'œ', 'p', 'ɕ', 'o', 'ŋ', 'aˤ', 'ɜ', 'ɒː', 'œː', 'eː', 'aː', 'd̠ʒ', 'uː', 'ɔː', 'oː', 'iː', 'θ', 'ʔ', 'ɛː', 'yː']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the dan language code:
Unknown phonemes: {'d': 57859, 'eˤ': 34287, 'ɔˤ': 26983, 'ð': 14994, 'ɑˤː': 12312, 'iˤ': 10617, 'uˤ': 9149, 'ɡ': 8951, 'b': 8373, 'oˤ': 6746, 'aˤ': 744, 'ɒː': 509, 'uː': 189, 'œː': 144, 'ɜ': 86, 'd̠ʒ': 69, 'ɔː'

# Norwegian analysis

We compare the Norwegian inventories in phoible to our Norwegian dataset. For Norwegian, we use phonemizer.

With our folding dictionary we get close to inventory 499.

Unseen phonemes:
* `ɑ` - not listed in the inventory, but listed in other inventories and on Wikipedia.
* `kː`, `pː`, `ɡː`, `bː`, `tː`, `dː` - long consonants produced by phonemizer. Not listed in phoible but is a feature of Norwegian (although perhaps not for `k` and `g`) 
* `ʂ`  - present in the other inventories and a valid consonant for Norwegian
* Remaining phonemes are rare or from loan words

Unknown phonemes:
* `æi` - diphthong that doesn't seem to be produced by our tool
* `ɳ`, `ɭ`, `ʈʰ` and `ɖ` - retroflex allophones of consonants we already have

In [25]:
compare_phoneme_sets('Norwegian', 'nob', ['Norwegian', 'NORWEGIAN'], 499)

Total phonemes in the dataset: 957505
Unique phonemes: ['t̪ʰ', 'ɑ', 'kː', 'WORD_BOUNDARY', 'ʋ', 'a', 'ɾ', 'ʃ', 'o̞ː', 'ɡ', 'uː', 'd̪', 'eː', 'e̞', 's', 'h', 'ʉː', 'tː', 'n̪', 'pː', 'ə', 'l', 'ɪ', 'b', 'iː', 'æ', 'j', 'kʰ', 'ʉ', 'ɒ̝', 'm', 'ø̞ː', 'f', 'yː', 'ai', 'pʰ', 'ŋ', 'dː', 'œ', 'øy', 'bː', 'ç', 'æː', 'ɑː', 'ʏ', 'æʉ', 'ʊ', 'ɡː', 'ɔy', 'ʂ', 'w', 'z']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the nob language code:
Unknown phonemes: {'ɑ': 48236, 'kː': 6345, 'tː': 6212, 'pː': 3562, 'ɡː': 1044, 'dː': 747, 'ʂ': 380, 'bː': 215, 'w': 19, 'z': 5}
Unseen phonemes: {'˥˧˥', '˩˨', 'ɭ̩', 'ɖ', 'ʈʰ', 'l̪̩', 'aː', 'ɳ', 'ɭ', 'ʔ', 'u', 'æi', 'l̪'

# Basque analysis

We compare the Basque inventories in phoible to our Basque dataset. For Basque, we use phonemizer.

The best inventory to match with the output of phonemizer seems to be 2161, since it is the only Basque inventory that contains diphthongs (which our tool produces and which exist in Basque). With our folding dictionary, we get a close match.

Unseen phonemes:
* `β`, `ð` and `ɣ` - the unvoiced `b`, `d` and `ɡ` which should be listed in the inventory
* `ɟ` - a valid consonant listed on Wikipedia and in other inventories
* `θ`  - rare, produced in Spanish loan words

Unknown phonemes:
* `˧˩` and `˧˥` - Pitch/tone is used to mark stress in some dialects of Basque, but our tool does not produce these


In [26]:
compare_phoneme_sets('Basque', 'eus', ['Basque', 'Zuberoan Basque', 'BASQUE'], 2161)

Total phonemes in the dataset: 1173225
Unique phonemes: ['e', 's̪̻', 'WORD_BOUNDARY', 'ɟ', 'ei̯', 't̺s̺', 'i', 'oi̯', 'a', 'ɾ', 'k', 't̠ʃ', 's̺', 'l', 'p', 'o', 'r', 't̪', 'u', 'ai̯', 'n', 'm', 'ð', 't̪̻s̪̻', 'β', 'ʎ', 'b', 'ɡ', 'ɣ', 'au̯', 'c', 'j', 'd̪', 'ʃ', 'ɲ', 'f', 'eu̯', 'θ', 'x', 'y']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the eus language code:
Unknown phonemes: {'β': 24938, 'ð': 22095, 'ɣ': 16526, 'θ': 264}
Unseen phonemes: {'ʒ', 's̻', 'ĩ', 'äː', 'l̪|l', 'r̺', '˧˥', 'i̯', 't̪|t', 'ʂ', 't̻s̻', 'n̪|n', 'l̺', 'iː', 'h', 'z̺', 'd̪|d', 'r̪|r', 'o̞ː', 'dz̻', 't', 'kʰ', 'tʰ', 'z̻', 'χ', 'n̺', 'h̃', 'ũ', 'uː', 'e̞ː', 'ɾ̪|ɾ', 

# Hungarian analysis

We compare the Hungarian inventories in phoible to our Hungarian dataset. For Hungarian, we use epitran.

With our folding dictionary, we get a good match with inventory 2191.

Unknown phonemes:
* `ŋ` - caused by nasal place assimilation, not always included in inventories but is valid

Unseen phonemes:
* `d̻z̻ː`, `ʒː`, `d̠ʒː` - long consonants that do not seem to be produced by our tool 

In [27]:
compare_phoneme_sets('Hungarian', 'hun', ['HUNGARIAN', 'Hungarian'], 2191)

Total phonemes in the dataset: 1155064
Unique phonemes: ['b', 'aː', 't̠ʃ', 'i', 'WORD_BOUNDARY', 'm', 'l̪', 'ɛ', 's̻', 'z̻', 'ɡ', 'k', 'o', 'd̪', 'ɑ', 'j', 'ø', 'n̪', 'ɲ', 'u', 't̻s̻', 't̪ː', 'y', 'r̪', 'h', 'v', 'd̠ʒ', 't̪', 'eː', 'oː', 'ʃ', 'ɟʝ', 's̻ː', 'p', 'z̻ː', 'uː', 'øː', 'n̪ː', 'f', 'l̪ː', 'iː', 'ɲː', 'ʃː', 'mː', 'r̪ː', 'kː', 't̠ʃː', 'jː', 'bː', 'cç', 't̻s̻ː', 'd̪ː', 'ɡː', 'ŋ', 'd̻z̻', 'pː', 'ʒ', 'vː', 'cçː', 'hː', 'fː', 'yː', 'ɟʝː', 'ʒː']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the hun language code:
Unknown phonemes: {'ŋ': 614}
Unseen phonemes: {'w', 'tː', 's̪', 'ɾ', 'äː', 'd̪z̪', 'dzː', 'dz', 's', 'l', 'c', 'ɔ', 'æ', 't

# Romanian analysis

We compare the Romanian inventories in phoible to our Romanian dataset. For Romanian, we use phonemizer.

With our folding dictionary, we get a close match with inventory 2443 (the other inventories do not include palatalised consonants, which phonemizer produces).

The main difference between the output of our tool and inventory 2443 is diphthongs. In Romanian, one view is that there are only two phonemic diphthongs, /oa/ and /ea/, however the traditional view is that there are many diphthongs, which phonemizer seems to produce. We could split the diphthongs into vowel glides, but we leave them for now. 

Unknown phonemes:
* `j`, `w` - valid approximant consonants that are listed in other inventories and Wikipedia but not this inventory
* `aɪ`, `uɪ`, `aʊ`, `eɪ`, `iɪ`, `əɪ`, `eʊ`, `iʊ`, `oʊ`, `əʊ`, `eo`, `oɪ`, `yɪ`, `äʊ` - valid Romanian diphthongs that our tool produces but aren't listed in the inventory. Strangely, only the falling diphthongs are produced (not the rising diphthongs).

Unseen phonemes:
* `i̯`, `u̯` - vowels listed in the inventory that are not produced by our tool. It is unclear these exist in the inventory since we already have `i` and `u`. 
* `ç` - not listed in other inventories and not produced by our tool.

In [28]:
compare_phoneme_sets('Romanian', 'ron', ['Rumanian', 'ROMANIAN', 'Romanian'], 2443)

Total phonemes in the dataset: 690134
Unique phonemes: ['m', 'ä', 'n̪', 'WORD_BOUNDARY', 'd̠ʒ', 'i', 'v', 'e̞', 'h', 'ʒ', 'd̪', 'o̞', 'l', 'u', 'ɾ̪', 't̠ʃ', 'p', 'j', 's̪', 'oʊ', 't̪', 'aɪ', 'k', 'w', 'ɡ', 'b', 't̠ʃʲ', 'ʃ', 'ʃʲ', 'ə', 'o̯ä', 'ɨ', 'uɪ', 'f', 'əɪ', 'eɪ', 'tsʲ', 'z̪', 't̪s̪', 'e̯ä', 'zʲ', 'iɪ', 'aʊ', 'nʲ', 'eʊ', 'tʲ', 'iʊ', 'ɾʲ', 'bʲ', 'mʲ', 'sʲ', 'kʲ', 'lʲ', 'eo', 'd̠ʒʲ', 'pʲ', 'dʲ', 'əʊ', 'fʲ', 'vʲ', 'oɪ', 'ʒʲ', 'ɡʲ']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the ron language code:
Unknown phonemes: {'aɪ': 6690, 'uɪ': 2377, 'aʊ': 1614, 'eɪ': 1377, 'iɪ': 1248, 'eʊ': 913, 'əɪ': 577, 'iʊ': 552, 'oʊ': 453, 'əʊ': 165, '

# Brazilian Portuguese analysis

We compare the Brazilian Portuguese inventories in phoible to our Brazilian Portuguese dataset. For Brazilian Portuguese, we use phonemizer.

With our folding dictionary, we get a close match with inventory 2207.

Unknown phonemes:
* `ɐ`, `õ`, `ɐ̃`, `ĩ`, `ũ` - vowels (mostly nasal) that exist in Brazilian portuguese but are not listed in the inventory (are listed in others)
* `w` - consonant that exists in Brazilian portuguese but isn't listed in this inventory (is listed in others)
* `d̠ʒ` - palatalised /d/ that isn't listed in the inventory but is valid in Brazilian portuguese

Unseen phonemes:
* `õɪ̯̃`, `ɔʊ̯`, `uʊ̯`, `ɐ̃ɪ̯̃`, `ũɪ̯̃`, `ʊ̯aɪ̯`, `ʊ̯ɐ̃`, - diphthongs that don't seem to be produced by our tool
* `ʎ` - consonant that doesn't seem to be produced by our tool

In [29]:
compare_phoneme_sets('PortugueseBr', 'por', ['Portuguese', 'Portuguese (Brazilian)'], 2207)

Total phonemes in the dataset: 752710
Unique phonemes: ['n̪', 'ɐ̃ʊ̯̃', 'WORD_BOUNDARY', 's̪', 'eɪ̯', 'd̠ʒ', 'i', 'a', 'd̪', 'ɐ', 'p', 'r', 'ɔ', 't̪', 'o', 'u', 'eʊ̯', 'm', 'k', 'ɾ', 'ɛ', 'l', 'ɐ̃', 'ẽɪ̯̃', 'ĩ', 'õ', 'e', 'oɪ̯', 'z', 'f', 'ɲ', 'v', 'ɡ', 'w', 'iʊ̯', 'ʒ', 'ʃ', 'oʊ̯', 'aʊ̯', 'b', 'ũ', 'aɪ̯', 'uɪ̯', 'ɣ', 'ɛʊ̯', 'ɔɪ̯', 'ɛɪ̯', 'ũɪ̯̃', 'ts̪', 'ɪ̯', 'ʎ', 'ə', 'ŋ']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the por language code:
Unknown phonemes: {'d̠ʒ': 6039, 'ɪ̯': 15, 'ts̪': 8}
Unseen phonemes: {'ĕ', 'ŭ', 'eu̜', 'ɐ̃ɪ̯̃', 'au̜', 'ɛu̜', 'ɐi', 's̻', 'i̯', 'x', 'ʁ', 's', 'oi', 'ĩ̯', 'ai', 'z̻', 'u̯', 'ui', 'ũi', 'ɔʊ̯', 

# Irish analysis

We compare the Irish inventories in phoible to our Irish dataset. For Irish, we use phonemizer.

With a substantial folding dictionary, we get close to inventory 2521. 

Unknown phonemes:
* `w`, `j` - voice approximate phonemes that are listed in other inventories for Irish but not this one. 
* The rest are sufficiently rare.

Unseen phonemes:
* `ŋ̟`,  `ʝ`, `ɟ` - consonants that our tool doesn't seem to produce
* `ɔʊ`, `uːə`, `ia` - dipthongs that our tool doesn't seem to produce (most of the others are)


# 

In [30]:
compare_phoneme_sets('Irish', 'gle', ['Irish', 'Irish Gaelic', 'IRISH'], 2521)

Total phonemes in the dataset: 444292
Unique phonemes: ['ə', 'n̪ˠ', 'WORD_BOUNDARY', 'w', 'ɪ', 'l̪ˠ', 'ʃ', 'ɛ̝', 'ɡ', 'ɾ̪ˠ', 'a', 'kʰ', 'mˠ', 'x', 'sˠ', 'd̪ˠ', 'bˠ', 'iː', 'pˠʰ', 'ɾ̪ʲ', 'ɔ̝', 'cʰ', 'eː', 't̪ˠʰ', 'ʊ', 'j', 'h', 'ɑː', 'fˠ', 'l̪ʲ', 'iːə', 'd̠ʒ', 'uːe', 'uː', 't̪ʲʰ', 'vˠ', 'oː', 'd̪ʲ', 'n̪ʲ', 'ŋ', 'i̞', 'ɣ', 'ç', 'ɐɪ', 'mʲ', 'ɐ', 'z', 'pʲʰ', 'fʲ', 'ʒ', 'χ', 'bʲ', 'vʲ', 'tʲ', 'xʲ', 'e']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the gle language code:
Unknown phonemes: {'z': 82, 'ɐ': 76, 'ʒ': 32, 'tʲ': 7, 'χ': 6, 'xʲ': 3}
Unseen phonemes: {'k', 'u', 'ɾʲ', 's', 'ɔ', 'ə̯', 'au', 'tʰ', 'kʲʰ', 'l̥ʲ', 't̪ˠ', 'ɡʲ', 'ɟʝ', 'ɾ̥ʲ', 

# Turkish analysis

We compare the Turkish inventories in phoible to our Turkish dataset. For Turkish, we use phonemizer.

With our folding dictionary, we get close to inventory 2217. 

Unknown phonemes are all sufficiently rare.

Unseen phonemes are just `ɣ` which is not always included in inventories.

In [31]:
compare_phoneme_sets('Turkish', 'tur', ['Turkish', 'TURKISH'], 2217)

Total phonemes in the dataset: 500533
Unique phonemes: ['a', 'j', 'WORD_BOUNDARY', 'ɟ', 'œ', 'ɾ', 'd̪', 'y', 'n̪', 'm', 'eː', 'd̠ʒ', 'i', 'o', 'u', 'k', 'l̪ˠ', 'ɯ', 'e', 'z̪', 'ɡ', 'b', 'ʃ', 's̪', 't̠ʃ', 'lʲ', 'iː', 'f', 't̪', 'p', 'h', 'v', 'aː', 'pː', 'bː', 'uː', 'ʒ', 'ɣ', 'c', 'ɡː', 'w']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the tur language code:
Unknown phonemes: {'pː': 24, 'w': 15, 'bː': 10, 'ɡː': 2}
Unseen phonemes: {'ɛː', 's̻', 'äː', 'l̪|l', 't̠ʃʰ', 'ø̞', 's', 'l', 'æ', 'o̞ː', 'kʰ', 'tʰ', 'z̻', 'd̻', 'e̞ː', 'ɪ', 'e̞', 'ʊ', 'n', 'ɫ̻', 'n̻', 't̻ʰ', 'cʰ', 'ʎ', 'r', 'ä', 'ʏ', 'o̞', 'œː', 'ʋ', 'd', 'ɛ', 'pʰ', 'l̻', 'ɯː', 'ɯ̞

# Quechua analysis

We compare the Quechua inventories in phoible to our Quechua dataset. For Quechua, we use phonemizer.

There are many Quechuan languages and many phoible inventories. With our folding dictionary, we get close to inventory 104, but there is still a lot of disagreement. 

Unknown phonemes:
* `d`, `g`, `f` - consonants that aren't listed in this inventory but are in others
* `aː` - a vowel not in this inventory but listed in others
* Remaining are sufficiently rare

There are several phonemes in inventory 104 that are not seen in our data, which could be an issue.

In [32]:
compare_phoneme_sets('Quechua', 'quh', ['Quechua', 'QUECHUA', 'Huallaga (Huanuco) Quechua', 'Ancash Quechua', 'Cajamarca Quechua', 'Cuzco-Collao Quechua', 'Ferreñafe Quechua', 'Imbabura Quichua', 'Jauja-Huanca Quechua', 'North Junín Quechua', 'Ayacucho Quechua', 'Bolivian Quechua', 'Huallaga Huánuco Quechua', 'Huaylas-Conchucos Quechua', 'San Martin Quechua', 'Yaru Quechua', 'Santiago del Estero Quechua', 'Salasca Quechua', 'Tena Quechua' ], 104)

Total phonemes in the dataset: 328326
Unique phonemes: ['d', 'aː', 'WORD_BOUNDARY', 't', 'a', 'ɪ', 'r', 't̠ʃ', 'n', 'm', 's', 'h', 'ɛ', 'p', 'ɔ', 'ʊ', 'β', 'ɡ', 'l', 'k', 'q', 'w', 'j', 'ʎ', 'pʼ', 'ʔ', 'kʼ', 'f', 'ɪː', 'tʼ', 't̠ʃʼ', 'qʼ', 'ʊː', 'ɛː']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the quh language code:
Unknown phonemes: {'aː': 1252, 'd': 560, 'ɡ': 411, 'f': 197, 'ɪː': 156, 'ɛː': 14, 'ʊː': 7}
Unseen phonemes: {'ʃ', 'ɾ', 'l̪|l', 't̠ʃʰ', 't̪|t', 'n̪|n', 'ɸ', 'r̪|r', 'ɣ', 'kʰ', 'tʰ', 't̪ʼ|tʼ', 'ɾ̪|ɾ', 'qʰ', 'ð', 's̪|s', 't̪ʰ|tʰ', 'pʰ', 'ɲ'}

Phoible inventories names with the quh language code: ['Quechua', 'QUECHUA']
Phoible 

# Farsi analysis

We compare the Farsi inventory in phoible to our Farsi dataset. For Farsi, we use phonemizer.

There are many Quechuan languages and many phoible inventories. With our folding dictionary, we get a very good match with inventory 516. There are no unknown phonemes, just the unseen phonemes `χ`, `ʒ` and `ʔ`.

In [33]:
compare_phoneme_sets('Farsi', 'pes', ['FARSI' ], 516)

Total phonemes in the dataset: 221955
Unique phonemes: ['b', 'a̟', 's', 'e', 'WORD_BOUNDARY', 'm', 'r', 'j', 'h', 't̠ʃ', 'kʰ', 'd̪', 'n̪', 'z', 'ʃ', 'ɡ', 'i', 'u', 'o', 'f', 't̪ʰ', 'ɑ', 'd̠ʒ', 'pʰ', 'l', 'w', 'ɢ', 'v', 'x', 'ʒ']

----------------------------------------------------------------------------------------------------
ALL PHOIBLE:
Comparing the phonemes seen in the data with all phoible phonemes:
Unknown phonemes: None
----------------------------------------------------------------------------------------------------
LANGUAGE CODE:

Comparing the phonemes seen in the data with just the phonemes in phoible with the pes language code:
Unknown phonemes: None
Unseen phonemes: {'o̞', 'd', 'ɢʁ', 'æ', 'ɣ', 'tʰ', 'ɒ', 'r̪', 't̠ʃʰ', 'χ', 'iː', 'uː', 'e̞', 'ɒː', 'l̪', 'ʔ', 'n'}

Phoible inventories names with the pes language code: ['Persian', 'FARSI', 'Farsi']
Phoible dialects names with the pes language code: [nan]
-------------------------------------------------------------------